# Landmarks Path Computation
In this notebook, we train emotion models using the CK+ dataset.
- Read landmarks files and convert it to a normalized space.
- TODO: PCA to avoid unnecessary landmarks
- Based on the normalized landmarks, compute its patch regresssion.
- 

## Function to read and pre-process features from a file
The dataset contains landmarks computed using AAM. These landmarks are stored as a sequence of (x \n y \n) floats dots. Each frame has its own landmark file. To achieve a normalized feature set, it is set the nose as the reference point and the distance between the eyes as the reference size. For each landmark, the final feature is its distance to the reference point divided by the reference size.

In [160]:
using Distances
function readLandmarksFile(filename)
    # println("Getting features from: ", filename)
    
    # Database parser
    dots = map((x)->parse(x), split(readall(filename), ['\n', ' '], limit=0, keep=false));
    getLandmarkPoint(id) = [dots[id*2-1], dots[id*2]];
    
    # Reference Point: nose
    referenceID = 34
    referencePoint = getLandmarkPoint(referenceID)
    
    # Reference Size: distance between eyes
    referenceSize = evaluate(Euclidean(), getLandmarkPoint(40), getLandmarkPoint(43))
    if referenceSize < 1.0 error("Distance between eyes less than one.") end
    
    # Normalized vector
    normalizedVector = []
    for id = 1:round(Int,(length(dots) / 2))
        if id == referenceID; continue end
        append!(normalizedVector, (referencePoint - getLandmarkPoint(id)) / referenceSize)
    end
    
    return normalizedVector
end;

## Function to perform patch regression over the landmarks
Using the above function, all the landmarks from a video sequence are read and normalized. Then, for each landmark trajectory, it is computed its patch regression. The regression uses a third order polinomial function: a + bx + cx^2 + dx^3. The resulting vector contains the parameters [x0,xt,a,b,c,d] for each landmark patch.

In [161]:
# returns patch_xy[file][landmark*2]
function getPatchFeatures(dir)
    landmarks4frame = []
    for f = readdir("$dir")
        push!(landmarks4frame, readLandmarksFile("$dir/$f"))
    end
    return landmarks4frame
end;

In [162]:
# returns the (x y) patch for a given landmark
function splitXY(patch, landmark)
    x, y = Float64[], Float64[]
    for file = 1:round(Int, length(patch))
        push!(x, patch[file][landmark*2-1])
        push!(y, patch[file][landmark*2])
    end
    return x, y
end;

In [163]:
# return abcd (cubic) params for a given (x, y) patch
using LsqFit
function cubicRegression(x, y)
    model(x, p) = p[1] + p[2] * x + p[3] * x .^ 2 + p[4] * x .^ 3;
    fit = curve_fit(model, x, y, [0.,0.,0.,0.]);
    return fit.param
end;

In [164]:
using Gadfly
function drawCubicCurve(x, y, param, title="no title")
    plot(layer(x=x, y=y, Geom.point, order=1),
         layer((x)->param[1] + param[2] * x + param[3] * x .^ 2 + param[4] * x .^ 3, minimum(x), maximum(x)),
         Guide.title(title))
end;

In [200]:
# performs regression for each landmark on a video clip
# returns [x0, xt, a, b, c, d] * lenght(landmarks), i.e. SVM ready
function landmarksRegressionParams(dir, debug=false)
    abcd = []
    patch = getPatchFeatures(dir)
    for landmark = 1:floor(Int, length(patch[1]) / 2)
        x, y = splitXY(patch, landmark)
        
        if debug 
            println("\n---\n $landmark")
            println("\n x: $(size(x))")
            show(x)
            println("\n y: $(size(y))")
            show(y)    
        end
        
        p = cubicRegression(x, y)
        push!(abcd, minimum(x))
        push!(abcd, maximum(x))
        append!(abcd, p)
        # if debug display(drawCubicCurve(x, y, p, "landmark: $landmark")) end
    end
    return abcd
end;

In [ ]:
p = landmarksRegressionParams("/home/data/ckplus/Landmarks/S100/001");
# dir = "/home/data/ckplus/Landmarks/$person/$shot"

In [ ]:
using Gadfly
function drawLandmarksCurves(param)
    layers = Array{Gadfly.Layer,1}[]
    for i in 1:floor(Int, length(param) / 6)
        x0, xt, a, b, c, d = param[i*6-5], param[i*6-4], param[i*6-3], param[i*6-2], param[i*6-1], param[i*6]
        push!(layers, layer((x)->a+b*x+c*x.^2+d*x.^3, x0, xt))
    end
    # TODO - a not so hardcoded revision
    plot( layers[1], layers[2], layers[3], layers[4], layers[5], layers[6], layers[7], layers[8], layers[9], layers[10], layers[11], layers[12], layers[13], layers[14], layers[15], layers[16], layers[17], layers[18], layers[19], layers[20], layers[21], layers[22], layers[23], layers[24], layers[25], layers[26], layers[27], layers[28], layers[29], layers[30], layers[31], layers[32], layers[33], layers[34], layers[35], layers[36], layers[37], layers[38], layers[39], layers[40], layers[41], layers[42], layers[43], layers[44], layers[45], layers[46], layers[47], layers[48], layers[49], layers[50], layers[51], layers[52], layers[53], layers[54], layers[55], layers[56], layers[57], layers[58], layers[59], layers[60], layers[61], layers[62], layers[63], layers[64], layers[65], layers[66], layers[67], Guide.title("Facial Landmarks Patchs"))
end;

In [ ]:
drawLandmarksCurves(p)

# Function to iterate over the CK+ dataset
For each sample, read the landmarks and the emotion result files. Using the above functions, the landmarks are transformed to an array of regression parameters. To perform the SVM trainning, the trainning matrix is composed by the regression parameters, where each row represent a sample video. In the same manner, the result matrix (for trainning) is composed by the emotion ID for the respective sample in the trainning matrix. So, for SVM trainning there are two matrixes, MTrainning: samples x (landmarks * regression_params), MTResults: samples x 1.

In [166]:
# read the sample file containing Emotion or FACS codes
function readResultFile(dir)
    results = []
    for a in readdir(dir)
        filename = "$dir/$a"
        append!(results, map((x)->convert(Int, parse(x)), split(readall(filename), ['\n', ' '], limit=0, keep=false)))
    end
    return(results)
end;
readResultFile("/home/data/ckplus/Emotion/S005/001")

1-element Array{Any,1}:
 3

In [190]:
# iterate over all samples to mount the trainning matrix for svm: X <samples> x <landmarks * regression_params>
function parseSVMTrainingX(dir)
    landmarksDir = "$dir/Landmarks"
    X = Matrix
    
    c, miss = 0, 0
    
    first = true
    for subjectDir = readdir(landmarksDir), sampleDir = readdir("$landmarksDir/$subjectDir")
        
        c += 1
        if c >= 10
            break
        end
        
        sampleDir = "$landmarksDir/$subjectDir/$sampleDir"
        println(sampleDir)
        
        try
            row = landmarksRegressionParams(sampleDir);
        catch
            println("$c Failed to perform landmark regression on $sampleDir")
        finally
            miss += 1
            continue
        end

        if first
            X = reshape(row, 1, size(row)[1])
        else
            vcat(X, reshape(row, 1, size(row)[1]))
        end
    end
    
    println("Total: $c, miss: $miss")
    
    return X
end
b = parseSVMTrainingX("/home/data/ckplus")

/home/data/ckplus/Landmarks/S005/001


0.024128534811045887 (predicted_residual) >
0.023175094108342526 (residual) + 3.469446951953614e-18 (eps)


/home/data/ckplus/Landmarks/S010/001


0.023136684282598047 (predicted_residual) >
0.023136356918004523 (residual) + 3.469446951953614e-18 (eps)
0.03560427176639378 (predicted_residual) >
0.035272337580467855 (residual) + 6.938893903907228e-18 (eps)
0.03525545868207217 (predicted_residual) >
0.03520573728277043 (residual) + 6.938893903907228e-18 (eps)
0.008916050423644146 (predicted_residual) >
0.006304918925433242 (residual) + 1.734723475976807e-18 (eps)


2 Failed to perform landmark regression on /home/data/ckplus/Landmarks/S010/001
/home/data/ckplus/Landmarks/S010/002
/home/data/ckplus/Landmarks/S010/003


...

/home/data/ckplus/Landmarks/S010/006


0.015851717963845463 (predicted_residual) >
0.010110986152224452 (residual) + 3.469446951953614e-18 (eps)
0.0002500275458194153 (predicted_residual) >
0.0002499399608806692 (residual) + 5.421010862427522e-20 (eps)


/home/data/ckplus/Landmarks/S011/001
8 Failed to perform landmark regression on /home/data/ckplus/Landmarks/S011/001
/home/data/ckplus/Landmarks/S011/002
Total: 10, miss: 9


0.00025022766747277725 (predicted_residual) >
0.0002500282244874036 (residual) + 5.421010862427522e-20 (eps)
0.0002502666891230047 (predicted_residual) >
0.00024994310354510216 (residual) + 5.421010862427522e-20 (eps)
0.0002502385885256351 (predicted_residual) >
0.00024999408221569407 (residual) + 5.421010862427522e-20 (eps)
0.0002501112713727199 (predicted_residual) >
0.00025002917504341425 (residual) + 5.421010862427522e-20 (eps)
0.0002501712000531751 (predicted_residual) >
0.00025005451767913197 (residual) + 5.421010862427522e-20 (eps)
0.0002501382798370097 (predicted_residual) >
0.0002500638083508149 (residual) + 5.421010862427522e-20 (eps)
0.00024981008885779324 (predicted_residual) >
0.0002497987587680746 (residual) + 5.421010862427522e-20 (eps)
0.0002498871567432716 (predicted_residual) >
0.00024981011754417177 (residual) + 5.421010862427522e-20 (eps)
0.0002499316571658581 (predicted_residual) >
0.00024988661836443035 (residual) + 5.421010862427522e-20 (eps)
0.000250033332007532

Array{T,2}

In [ ]:
using RegERMs
function trainCKPlus(dir, debug = false)
    landmarksDir = "$dir/Landmarks"
    emotionDir = "$dir/Emotion"
    facsDir =  "$dir/FACS"
end

trainCKPlus("/home/data/ckplus", true)

In [ ]:
# "/home/data/ckplus/Emotion/S005/001/S005_001_00000011_emotion.txt"
593 * 67 * 6

In [201]:
landmarksRegressionParams("/home/data/ckplus/Landmarks/S010/003", true)


---
 1


0.00013743956508164825 (predicted_residual) >
0.0001363609285362702 (residual) + 2.710505431213761e-20 (eps)


LoadError: LoadError: Base.LinAlg.SingularException(4)
while loading In[201], in expression starting on line 1


 x: (18,)
[1.6125587940086228,1.613146336527954,1.613854472054303,1.6133666499119552,1.6127935733958823,1.61212187727935,1.6124093097427878,1.6091185921922317,1.6021711990252059,1.597788682405467,1.5933943218193074,1.5875754859571956,1.5855649748971035,1.5842655269915151,1.5814208079571512,1.5796968709963122,1.5793581788994282,1.5782900635363626]
 y: (18,)
[1.2005137495649008,1.200579293024512,1.199354687828176,1.200340110482455,1.200074351804677,1.1988383146913537,1.1967784285754253,1.1985410604529516,1.205598360991823,1.2125404855621087,1.2114197271657834,1.215806853648853,1.2175258514318594,1.2185035380367226,1.2207190694021894,1.2220891017034743,1.2224178256647364,1.2220027536117763]
---
 2

 x: (18,)
[1.6159705337023482,1.6166308170154187,1.6169828221010247,1.6166814591766943,1.6157209961188022,1.6141357932063256,1.613128939919174,1.6098342799094572,1.6044844034210308,1.6018354758234399,1.5962652568196567,1.5912300027241488,1.5895585292871994,1.5884453999048824,1.5860581067617574

0.0001390276926636207 (predicted_residual) >
0.00013666117198592713 (residual) + 2.710505431213761e-20 (eps)
0.00022338284059070733 (predicted_residual) >
0.00022090922661087284 (residual) + 2.710505431213761e-20 (eps)
0.0002201705845372338 (predicted_residual) >
0.00022016713092219746 (residual) + 2.710505431213761e-20 (eps)
0.00022017088254095793 (predicted_residual) >
0.00022016855972893589 (residual) + 2.710505431213761e-20 (eps)
0.0002201670154172418 (predicted_residual) >
0.00022016653371512398 (residual) + 2.710505431213761e-20 (eps)
0.000471812916442793 (predicted_residual) >
0.0004393896062444279 (residual) + 5.421010862427522e-20 (eps)
0.00044551929765171617 (predicted_residual) >
0.00044099394062964425 (residual) + 5.421010862427522e-20 (eps)
0.0004546884494184248 (predicted_residual) >
0.0004433567234535528 (residual) + 5.421010862427522e-20 (eps)
0.0004656921980896637 (predicted_residual) >
0.00044991247081704996 (residual) + 5.421010862427522e-20 (eps)
0.00045432585146537